# Import packages

In [5]:
import os, wget, zipfile

# Download and extract data

In [6]:
def download_data(destination_dir, url="https://archive.ics.uci.edu/ml/machine-learning-databases/00552/data.zip"):

    # Make path if does not exits
    if(os.path.exists(destination_dir) == False):
        os.makedirs(destination_dir)
    # Download data if it does not exist on disk
    if(os.path.exists(os.path.join(destination_dir, "data.zip")) == False):
        wget.download(url, destination_dir)
        
def extract_data(data_dir, filename="data.zip"):
    
    with zipfile.ZipFile(os.path.join(data_dir, filename), 'r') as zip_ref:
        zip_ref.extractall(data_dir)

In [7]:
root_dir = "/mnt/Disk-2/My Stuff/UWaterloo/Coursework/Spring'22/ECE 659/Project/activity-recognition/"
raw_data_dir = os.path.join(root_dir, 'data/raw')
download_data(raw_data_dir)
extract_data(raw_data_dir)

100% [................................................................................] 1117 / 1117